<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#The-analysis" data-toc-modified-id="The-analysis-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>The analysis</a></span></li><li><span><a href="#Load-libraries" data-toc-modified-id="Load-libraries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load libraries</a></span></li><li><span><a href="#Import-datasets" data-toc-modified-id="Import-datasets-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Import datasets</a></span><ul class="toc-item"><li><span><a href="#Worldometer-data-and-WHO-data-for-filling-regions" data-toc-modified-id="Worldometer-data-and-WHO-data-for-filling-regions-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Worldometer data and WHO data for filling regions</a></span></li><li><span><a href="#Happiness-report-datasets" data-toc-modified-id="Happiness-report-datasets-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Happiness report datasets</a></span></li></ul></li><li><span><a href="#Palettes-and-categories" data-toc-modified-id="Palettes-and-categories-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Palettes and categories</a></span><ul class="toc-item"><li><span><a href="#Regions" data-toc-modified-id="Regions-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Regions</a></span></li><li><span><a href="#Variable-group" data-toc-modified-id="Variable-group-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Variable group</a></span></li></ul></li><li><span><a href="#Analyse-correlation" data-toc-modified-id="Analyse-correlation-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Analyse correlation</a></span></li><li><span><a href="#Analyse-the-score-influence-on-the-pandemic-progress" data-toc-modified-id="Analyse-the-score-influence-on-the-pandemic-progress-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Analyse the score influence on the pandemic progress</a></span></li><li><span><a href="#Focus-on-significant-correlations" data-toc-modified-id="Focus-on-significant-correlations-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Focus on significant correlations</a></span></li></ul></div>

# The analysis 

# Load libraries

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
#import colours as c
import re

# Import datasets

## Worldometer data and WHO data for filling regions

In [2]:
file = open('datasets/worldometer_data.csv')
worldometer = pd.read_csv('datasets/worldometer_data.csv')
file.close()
worldometer.fillna(0, inplace=True)
worldometer.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,3.315104e+08,7667817,30905.0,214884.0,273.0,4874445.0,25407.0,2578488.0,14200.0,23130.0,648.0,112261589.0,338637.0,Americas
1,India,Asia,1.383530e+09,6682073,59893.0,103600.0,886.0,5659110.0,75657.0,919363.0,8944.0,4830.0,75.0,79982394.0,57810.0,South-EastAsia
2,Brazil,South America,2.129539e+08,4918022,2733.0,146417.0,42.0,4263208.0,0.0,508397.0,8318.0,23094.0,688.0,17900000.0,84056.0,Americas
3,Russia,Europe,1.459510e+08,1225889,10888.0,21475.0,117.0,982324.0,3181.0,222090.0,2300.0,8399.0,147.0,48042343.0,329168.0,Europe
4,Colombia,South America,5.102364e+07,855052,0.0,26712.0,0.0,761674.0,0.0,66666.0,2220.0,16758.0,524.0,3894289.0,76323.0,Americas


In [3]:
china=["China", "Asia",
       1439323776, 85470, 20, 4634, 0, 80628, 7, 208, 2, 59, 3, 160000000, 111163, 'Western Pacific']
worldometer.loc[len(worldometer)] = china

In [4]:
l = []
for i in worldometer["WHO Region"]:
    if i != 0:
        l.append(re.sub(r"(\w)([A-Z])", r"\1 \2", i))
    else:
        l.append(i)
worldometer["WHO Region"] = l

In [5]:
w_regions = worldometer[worldometer["WHO Region"]!=0][["WHO Region", "Country/Region"]]

In [6]:
file = open('datasets/full_grouped.csv')
full_grouped = pd.read_csv('datasets/full_grouped.csv')
file.close()
full_grouped.fillna(0, inplace=True)
full_grouped.head()

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,WHO Region
0,2020-01-22,Afghanistan,0,0.0,0,0.0,0,0,0,Eastern Mediterranean
1,2020-01-22,Albania,0,0.0,0,0.0,0,0,0,Europe
2,2020-01-22,Algeria,0,0.0,0,0.0,0,0,0,Africa
3,2020-01-22,Andorra,0,0.0,0,0.0,0,0,0,Europe
4,2020-01-22,Angola,0,0.0,0,0.0,0,0,0,Africa


In [7]:
full_grouped.replace(to_replace='US', value='USA', regex=True, inplace=True)

I make a merged dataframe with consecutive WHO Regions - this way I fill wordometer data.

In [8]:
new_df = pd.concat([full_grouped[["Country/Region", "WHO Region"]], w_regions]).drop_duplicates().reset_index(drop=True)
new_df.head()

,Country/Region,WHO Region
0,Afghanistan,Eastern Mediterranean
1,Albania,Europe
2,Algeria,Africa
3,Andorra,Europe
4,Angola,Africa


In [9]:
worldometer = worldometer.drop("WHO Region", 1)
worldometer = pd.merge(worldometer, new_df, on='Country/Region')
worldometer["WHO Region"].unique()

array(['Americas', 'South-East Asia', 'Europe', 'Africa',
       'Eastern Mediterranean', 'Western Pacific'], dtype=object)

In [10]:
worldometer["Recovered/1M pop"] = worldometer[["TotalRecovered"]].div(worldometer["Population"], axis=0)*10**6

worldometer["Recovered/1M cases"] = worldometer[["TotalRecovered"]].div(worldometer["TotalCases"], axis=0)*10**6
worldometer["Deaths/1M cases"] = worldometer[["TotalDeaths"]].div(worldometer["TotalCases"], axis=0)*10**6
worldometer["Confirmed/1M tests"] = worldometer[["TotalCases"]].div(worldometer["TotalTests"], axis=0)*10**6

In [11]:
worldometer.shape

(183, 20)

In [12]:
worldometer.describe()

,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Recovered/1M pop,Recovered/1M cases,Deaths/1M cases,Confirmed/1M tests
count,1.830000e+02,1.830000e+02,183.000000,183.000000,183.000000,1.830000e+02,183.00000,1.830000e+02,183.000000,183.000000,183.000000,1.830000e+02,1.830000e+02,183.000000,183.000000,183.000000,1.830000e+02
mean,4.244545e+07,1.944830e+05,1103.841530,5705.786885,16.131148,1.435048e+05,885.04918,3.698716e+04,364.256831,5484.852459,129.575301,3.685808e+06,1.323357e+05,4143.981267,758710.846640,24972.103881,inf
std,1.521722e+08,8.408190e+05,5149.942004,22098.068329,73.207400,6.405577e+05,5934.60739,2.085873e+05,1468.424926,7658.500215,209.713583,1.604706e+07,2.460206e+05,6598.291669,222380.812223,27416.843953,NaN
min,3.395000e+04,1.400000e+01,0.000000,0.000000,0.000000,0.000000e+00,0.00000,0.000000e+00,0.000000,3.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,4.173471e+02
25%,2.759925e+06,3.132500e+03,0.000000,54.000000,0.000000,1.660500e+03,0.00000,2.700000e+02,0.000000,522.500000,8.000000,5.479100e+04,1.108350e+04,312.346087,656473.910246,10687.165847,1.723688e+04
50%,9.921338e+06,1.452700e+04,28.000000,261.000000,0.000000,8.308000e+03,2.00000,1.981000e+03,5.000000,2698.000000,40.000000,2.966110e+05,4.869300e+04,1474.474308,826229.730187,19241.126496,6.074874e+04
75%,3.135696e+07,8.458300e+04,445.500000,1532.500000,7.000000,5.320050e+04,273.00000,1.095400e+04,106.500000,7684.000000,142.500000,1.529686e+06,1.485805e+05,4686.575313,922938.720896,30741.337547,1.696575e+05
max,1.439324e+09,7.667817e+06,59893.000000,214884.000000,886.000000,5.659110e+06,75657.00000,2.578488e+06,14200.000000,45121.000000,1237.000000,1.600000e+08,1.778274e+06,44042.944578,1000000.000000,289563.939245,inf


In [13]:
dif = worldometer[worldometer["TotalCases"] != worldometer["TotalRecovered"]+worldometer["TotalDeaths"]+worldometer["ActiveCases"]]

In [ ]:
dif["difference"] = dif["TotalCases"] - (dif["TotalRecovered"]+dif["TotalDeaths"]+dif["ActiveCases"])

In [15]:
dif

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,...,Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region,Recovered/1M pop,Recovered/1M cases,Deaths/1M cases,Confirmed/1M tests,difference
5,Spain,Europe,46759558.0,852838,2099.0,32225.0,46.0,0.0,0.0,0.0,...,18239.0,689.0,13689776.0,292770.0,Europe,0.0,0.0,37785.605238,62297.440075,820613.0
11,UK,Europe,67979397.0,515571,12594.0,42369.0,19.0,0.0,0.0,0.0,...,7584.0,623.0,25865851.0,380495.0,Europe,0.0,0.0,82178.788179,19932.497098,473202.0
27,Netherlands,Europe,17144850.0,140471,4579.0,6461.0,7.0,0.0,0.0,0.0,...,8193.0,377.0,2603166.0,151834.0,Europe,0.0,0.0,45995.258808,53961.599068,134010.0
41,Sweden,Europe,10115730.0,94283,0.0,5895.0,0.0,0.0,0.0,0.0,...,9320.0,583.0,1661484.0,164248.0,Europe,0.0,0.0,62524.527221,56746.258164,88388.0


In [16]:
dif["difference"].sum()

1516213.0

## Happiness report datasets

In [17]:
differences_w = ['Trinidad and Tobago','UK','Congo','USA','Czechia','UAE','S. Korea','Palestine',]
differences_d = ['Trinidad & Tobago','United Kingdom','Congo (Brazzaville)','United States','Czech Republic',
                 'United Arab Emirates','South Korea','Palestinian Territories',]

In [18]:
file = open('datasets/2019.csv')
d19 = pd.read_csv('datasets/2019.csv')
file.close()
d19.columns = ["rank", "Country/Region", "Score", "GDP", "Family", "Life expectancy", "Freedom", "Generosity", "Trust"]
d19.drop(columns=["rank"], inplace=True)
d19.replace(to_replace=differences_d, value=differences_w, regex=True, inplace=True)
d19.head()

,Country/Region,Score,GDP,Family,Life expectancy,Freedom,Generosity,Trust
0,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393
1,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410
2,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341
3,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118
4,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298


In [19]:
df = pd.merge(worldometer, d19, on='Country/Region')

In [20]:
df

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,...,Recovered/1M cases,Deaths/1M cases,Confirmed/1M tests,Score,GDP,Family,Life expectancy,Freedom,Generosity,Trust
0,USA,North America,3.315104e+08,7667817,30905.0,214884.0,273.0,4874445.0,25407.0,2578488.0,...,635701.791005,28024.142986,68303.121916,6.892,1.433,1.457,0.874,0.454,0.280,0.128
1,India,Asia,1.383530e+09,6682073,59893.0,103600.0,886.0,5659110.0,75657.0,919363.0,...,846909.334873,15504.170637,83544.298511,4.015,0.755,0.765,0.588,0.498,0.200,0.085
2,Brazil,South America,2.129539e+08,4918022,2733.0,146417.0,42.0,4263208.0,0.0,508397.0,...,866854.194634,29771.521966,274749.832402,6.300,1.004,1.439,0.802,0.390,0.099,0.086
3,Russia,Europe,1.459510e+08,1225889,10888.0,21475.0,117.0,982324.0,3181.0,222090.0,...,801315.616667,17517.899255,25516.844589,5.648,1.183,1.452,0.726,0.334,0.082,0.031
4,Colombia,South America,5.102364e+07,855052,0.0,26712.0,0.0,761674.0,0.0,66666.0,...,890792.606765,31240.205274,219565.625458,6.125,0.985,1.410,0.841,0.470,0.099,0.034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,Mongolia,Asia,3.291861e+06,314,1.0,0.0,0.0,307.0,0.0,7.0,...,977707.006369,0.000000,4263.929061,5.285,0.948,1.531,0.667,0.317,0.235,0.038
145,Bhutan,Asia,7.738120e+05,298,15.0,0.0,0.0,237.0,7.0,61.0,...,795302.013423,0.000000,2091.081328,5.082,0.813,1.321,0.604,0.457,0.370,0.167
146,Cambodia,Asia,1.677868e+07,280,2.0,0.0,0.0,275.0,0.0,5.0,...,982142.857143,0.000000,1942.838904,4.700,0.574,1.122,0.637,0.609,0.232,0.062
147,Laos,Asia,7.302776e+06,23,0.0,0.0,0.0,22.0,0.0,1.0,...,956521.739130,0.000000,417.347124,4.796,0.764,1.030,0.551,0.547,0.266,0.164


# Palettes and categories

I use divik package to have consistent color theme for all palettes, no matter their length.

In [21]:
from ast import literal_eval as make_tuple
from divik._inspect.color import make_colormap

In [22]:
def make_palette(length):
    l = [i for i in range(length)]
    p = make_colormap(l)
    palette = []
    for i in p:
        palette.append(i[1])
    return palette

## Regions

This palette provides colors for:

In [23]:
worldometer["WHO Region"].unique()

array(['Americas', 'South-East Asia', 'Europe', 'Africa',
       'Eastern Mediterranean', 'Western Pacific'], dtype=object)

In [24]:
region_palette = make_palette(len(worldometer["WHO Region"].unique()))

## Variable group

In [25]:
var_palette = ["1.", "2.", "3."]

# Analyse correlation

In [26]:
from scipy.stats import pearsonr

In [27]:
def pearsonr_pval(x,y):
    return pearsonr(x,y)[1]

In [28]:
to_use=[['Tot Cases/1M pop', 'Deaths/1M pop', 'Tests/1M pop', 'Recovered/1M pop'],
       ['Recovered/1M cases', 'Deaths/1M cases', 'Confirmed/1M tests'],
       ['GDP', 'Family', 'Life expectancy', 'Freedom', 'Generosity', 'Trust']]

In [29]:
def update_corr(cval, pval):
    if pval > 0.05:
        return 0
    return cval

In [30]:
def correlo(df, to_use, point_size, palette, title, height, width, fontsize, var_palette):
    multicat = [var_palette[to_use.index(sublist)] for (sublist) in to_use for item in sublist]
    dh = df[[item for sublist in to_use for item in sublist]]
    corr = dh.corr()
    pval = dh.corr(method=pearsonr_pval)
    corr_list = [j for sub in (corr.values.tolist()) for j in sub]
    pval_list =  [j for sub in (pval.values.tolist()) for j in sub]
    corr_list = [update_corr(cval, pval) for cval, pval in zip(corr_list, pval_list)]
    
    x = [[i for _ in range(corr.shape[0])] for i in corr.index]
    mx = [[i for _ in range(corr.shape[0])] for i in multicat]
    x = [[j for sub in (mx) for j in sub],[j for sub in (x) for j in sub]]
    y = [multicat* corr.shape[1], [i for i in corr.index] * corr.shape[1]]

    size = [j*point_size if j > 0 else -j*point_size for j in corr_list] 
    # here another if making it 0 for small p-values
    colors = corr_list

    fig = go.Figure(data=go.Scatter(
        y=y,
        x=x,
        mode='markers',
        hovertext=[i if i != 0 else "" for i in colors],
        hoverinfo="text",
        marker=dict(size=size,
                    color=colors,
                    colorbar=dict(title="Correlation"),
                    colorscale=palette, cmid=0)))   

    fig.update_layout(
        height=height, width=width, font=dict(size=fontsize), title=title)
    fig.update_xaxes(
        range=[-0.5, corr.shape[0]-0.5],
        showgrid=True,
        ticks="outside",
        tickson="boundaries",
        ticklen=0)
    fig.update_yaxes(
        range=[-0.5, corr.shape[0]-0.5],
        showgrid=True,
        ticks="outside",
        tickson="boundaries",
        ticklen=0)
    return fig

In [31]:
for_correlo = df.dropna()

In [34]:
fig= correlo(for_correlo, to_use=to_use, point_size=46, palette="RdBu_r", 
             title="Correlogram of disease and country specifics: "+str(df.shape[0])+ " countries",
             height=885, width=885, fontsize=14, var_palette=var_palette)
fig.show()

In [35]:
for region in for_correlo["WHO Region"].unique():
    regional = for_correlo[for_correlo["WHO Region"].isin([region])]
    fig = correlo(regional, to_use, 46, "RdBu_r", region + ": "+str(regional.shape[0])+ " countries", 
                  height=885, width= 885, fontsize=14, var_palette=var_palette)
    fig.show()

# Analyse the score influence on the pandemic progress

In [ ]:
def basic_violins(df, plot_col, val, title, info, height=500, width=985, to_sort=True,
                  palette=px.colors.qualitative.Plotly):
    regions = df[plot_col].unique()
    j = 0
    for_sort = []
    data = []
    q = []
    fig = go.Figure()
    for region in regions:
        x = df[plot_col][(df[plot_col] == region)]
        y = df[val][(df[plot_col] == region)]
        quantity = x.shape[0]
        if quantity > 2:
            for_sort.append(y.median())
            data.append(go.Violin(x=x,
                                  text=df[info][(df[plot_col] == region)],
                                  y=y,
                                  name=region,
                                  marker=dict(color=palette[j]),
                                  box_visible=True,
                                  meanline_visible=True
                                  ))
        q.append(region + ":<br>" + str(quantity))
        j += 1
    if not to_sort:
        for trace in range(len(data)):
            fig.add_trace(data[trace])
    else:
        for trace in range(len(data)):
            indices = np.argsort(for_sort)
            fig.add_trace(data[indices[trace]])
    fig.update_xaxes(ticktext=q, tickvals=regions, title=plot_col)
    fig.update_yaxes(title=val)
    fig.update_layout(height=height, width=width, title_text=title)
    return fig


# Focus on significant correlations